<a href="https://colab.research.google.com/github/SeanBarnier/HAFS_Air-Sea/blob/main/fluxes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set up environment

In [ ]:
!pip install cfgrib

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import cfgrib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#User parameters

In [ ]:
name = "Milton"
tcNum = "14"
trackType = ""

initTime = dt(year=2024, month=10, day=6, hour=0) #Time when Milton began its most rapid intensification

fHourStep = 6       #Normally 3 for HAFS-A
forecastLength = 48 #Normally 126 for HAFS-A.
#runStep = 6         #Normally 6 for HAFS-A

figureSuffix = "_RI"
subfolder = "RI/"
dataPath = "/content/drive/MyDrive/savedData/"
figurePath = "/content/drive/MyDrive/figures/"

stormCentered = True
potentialTemp = True #Use atmospheric potential temperature instead of in-situ temperature

atmTop = 850
oceFloor = 100

Get ATCF data and find interested point

#Retrieve HAFS-A Data

Find times needed

In [ ]:
dateFormat = "%Y-%m-%d %H:%M:%S"
runFormat = "%Y%m%d%H"

fcastTimes = [] #Key: initiation, item: valid time list
fhour = 0
validTime = initTime

while fhour <= forecastLength:
    fcastTimes.append(validTime)
    validTime += pd.Timedelta(hours=fHourStep)
    fhour += fHourStep

Find storm location in HAFS-A from ATCF files. Used to find along-storm profile.

In [ ]:
cols = ["BASIN", "CY", "YYYYMMDDHH", "TECHNUM/MIN", "TECH", "TAU", "LatN/S", "LonE/W",
    "VMAX", "MSLP", "TY", "RAD", "WINDCODE", "RAD1", "RAD2", "RAD3", "RAD4",
    "POUTER", "ROUTER", "RMW", "GUSTS", "EYE", "SUBREGION", "MAXSEAS", "INITIALS",
    "DIR", "SPEED", "STORMNAME", "DEPTH", "SEAS", "SEASCODE", "SEAS1", "SEAS2",
    "SEAS3", "SEAS4", "USERDEFINED1", "Thermo1", "Thermo2", "Thermo3", "Thermo4",
    "Thermo5", "Thermo6", "Thermo7", "USERDEFINED2", "DT", "SHR82", "SHR81_1",
    "SHR82_2",  "USERDEFINED3", "SST", "USERDEFINED4", "ARMW1", "ARMW2"]

initStr, initHour = initTime.strftime("%Y%m%d_%H").split("_")

atcfURL = f"https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/{initStr}/{initHour}/{tcNum}l.{initStr}{initHour}.hfsa.trak.atcfunix"
atcfFile = "atcf_" + initStr + "_" + initHour + ".csv"

!wget -O {atcfFile} {atcfURL}
atcf = pd.read_csv(atcfFile, names=cols)

In [ ]:
tcLocs = {}

for valid in fcastTimes:
    fHour = int((valid-initTime).total_seconds() / 3600)

    pointLat = int(atcf[atcf.TAU==fHour]["LatN/S"].iloc[0].replace("N", ""))/10
    pointLon = int(atcf[atcf.TAU==fHour]["LonE/W"].iloc[0].replace("W", ""))/-10 #Assume western hemisphere
    tcLocs[valid] = (pointLat, pointLon)

Get data from HAFS-A output.

In [ ]:
fcastTimes

In [ ]:
atmData

In [ ]:
atm = {}

for valid in fcastTimes:

  initStr = initTime.strftime("%Y%m%d%H")

  atm[valid] = {}

  fhour = str(int((valid-initTime).total_seconds() / 3600))
  while len(fhour) < 3: fhour = "0" + fhour

  atmFile = "hafsa_" + initStr + "_f" + fhour + ".nc"
  atmPath = dataPath + "hafsaOutput/" + subfolder + atmFile
  atmData = xr.open_dataset(atmPath)

  point = tcLocs[valid]
  layer = max(atmData.isobaricInhPa.data) #Get lowest point; should be 1000 hPa

  #Longitude in atm files are in degrees east, but are -180 - 180 in oce files. point has them from -180 - 180
  validPoint = atmData.sel(latitude=point[0], longitude=point[1] + 360, isobaricInhPa=layer, method="nearest")

  atm[valid]["T"] = float(validPoint.t.data)
  atm[valid]["q"] = float(validPoint.q.data) * 1000 #Convert from kg/kg to g/kg
  atm[valid]["u"] = float(validPoint.u.data)
  atm[valid]["v"] = float(validPoint.v.data)
  atm[valid]["gh"] = float(validPoint.gh.data)
  atm[valid]["sst"] = float(validPoint.sst.data)
  atm[valid]["shf"] = float(validPoint.ishf.data)
  atm[valid]["lhf"] = float(validPoint.slhtf.data)

In [ ]:
oce = {}

for valid in fcastTimes:

  fhour = str(int((valid-initTime).total_seconds() / 3600))
  while len(fhour) < 3: fhour = "0" + fhour
  oceFile = "mom6_" + initStr + "_f" + fhour + ".nc"
  ocePath = dataPath + "mom6Output/" + subfolder + oceFile

  if oceFile == 'mom6_2024100800_f000.nc': #This file is missing
    for layer in oceData.z_l.data:
      oce[initTime][valid][layer] = {"T":np.nan, "s":np.nan, "u":np.nan, "v":np.nan}
    continue

  oceData = xr.open_dataset(ocePath, decode_times=False)

  point = tcLocs[valid]
  layer = min(oceData.z_l.data[oceData.z_l.data<=oceFloor]) #Retrieves most shallow layer; should be 1 m

  oce[valid] = {}
  oce[valid]["T"] = float(oceData.sel(yq=point[0], yh=point[0], xh=point[1], xq=point[1], z_l=layer, method="nearest").temp.data) + 273.15 #This is potential temperature. Converted from C to K.
  oce[valid]["s"] = float(oceData.sel(yq=point[0], yh=point[0], xh=point[1], xq=point[1], z_l=layer, method="nearest").so.data)
  oce[valid]["u"] = float(oceData.sel(yq=point[0], yh=point[0], xh=point[1], xq=point[1], z_l=layer, method="nearest").uo.data)
  oce[valid]["v"] = float(oceData.sel(yq=point[0], yh=point[0], xh=point[1], xq=point[1], z_l=layer, method="nearest").vo.data)
  oce[valid]["sst"] = float(oceData.sel(yq=point[0], yh=point[0], xh=point[1], xq=point[1], z_l=layer, method="nearest").SST.data) + 273.15
  oce[valid]["shf"] = float(oceData.sel(yq=point[0], yh=point[0], xh=point[1], xq=point[1], z_l=layer, method="nearest").sensible.data)
  oce[valid]["lhf"] = float(oceData.sel(yq=point[0], yh=point[0], xh=point[1], xq=point[1], z_l=layer, method="nearest").latent.data)

#Figures

*Why are the HAFS-A and MOM6 fluxes different?*

In [ ]:
compFig = plt.figure(figsize=(10, 10))
sstAx = compFig.add_axes([0.1, 0.7, 0.8, 0.25])
shfAx = compFig.add_axes([0.1, 0.4, 0.8, 0.25])
lhfAx = compFig.add_axes([0.1, 0.1, 0.8, 0.25])

atmSST = [atm[valid]['sst'] for valid in atm.keys()]
atmSHF = [atm[valid]['shf']*-1 for valid in atm.keys()]
atmLHF = [atm[valid]['lhf']*-1 for valid in atm.keys()]

oceSST = [oce[valid]['sst'] for valid in oce.keys()]
oceSHF = [oce[valid]['shf'] for valid in oce.keys()]
oceLHF = [oce[valid]['lhf'] for valid in oce.keys()]

sstAx.plot(atm.keys(), atmSST, label="Atm. SST", color="red")
sstAx.plot(oce.keys(), oceSST, label="Oce. SST", color="blue")
shfAx.plot(atm.keys(), atmSHF, label="Atm. SHF", color="red")
shfAx.plot(oce.keys(), oceSHF, label="Oce. SHF", color="blue")
lhfAx.plot(atm.keys(), atmLHF, label="Atm. LHF", color="red")
lhfAx.plot(oce.keys(), oceLHF, label="Oce. LHF", color="blue")

sstAx.set_title("SST")
shfAx.set_title("Sensible Heat Flux")
lhfAx.set_title("Latent Heat Flux")

sstAx.legend()
shfAx.legend()
lhfAx.legend()